In [66]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [67]:
# Libraries used on the tutorial
import boto3
import os
import time
import pandas as pd
import matplotlib as plt
from botocore.exceptions import ClientError
from datetime import date
import json
import seaborn as sns
import spacy
import numpy as np
#import wmd
from gensim.models import word2vec


%matplotlib inline

In [100]:
# Create a dictionary with the URLs for each speech on Youtube
dict_urls_youtube = {
    '2020-04-05':'https://www.youtube.com/watch?v=jJSfz7POzM8',
    '2020-04-02':'https://www.youtube.com/watch?v=BbWEVgbIXbM'
   }

In [101]:
# Download each speech in mp3 format using Youtube-DL
for url in list(dict_urls_youtube.values()):
    print(f'Downloading audio file from the link: {url}')
    os.system(f'youtube-dl -x --audio-format "mp3" {url}')

In [102]:
# Create a Dataframe to store the information trough the analysis
df_audio = pd.DataFrame.from_dict(dict_urls_youtube, orient='index', columns=['youtube_urls'])
df_audio

,youtube_urls
2020-04-05,https://www.youtube.com/watch?v=jJSfz7POzM8
2020-04-02,https://www.youtube.com/watch?v=BbWEVgbIXbM


In [103]:
os.listdir('.')

["'Berlin Wall' Speech - President Reagan's Address at the Brandenburg Gate - 6_12_87-5MDFX-dNtsM.m4a",
 "'We choose to go to the moon' Speech by President John F. Kennedy-QAmHcdwKgtQ.m4a",
 '.anaconda',
 '.aws',
 '.azuredatastudio',
 '.boto',
 '.cache',
 '.conda',
 '.condarc',
 '.config',
 '.dotnet',
 '.gitconfig',
 '.idlerc',
 '.ipynb',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.matplotlib',
 '3D Objects',
 '558_May_7_320-1.ipynb',
 'AppData',
 'Application Data',
 'AudioToText.ipynb',
 'AudioToTextMedical_3.ipynb',
 'AudioToTextMedical_4.ipynb',
 'AudioToText_Nokey.ipynb',
 'AudioToText_youtube.ipynb',
 'Audio_onesample.ipynb',
 'Audio_To_TextMedical_short.ipynb',
 'Contacts',
 'Cookies',
 'Desktop',
 'Documents',
 'Downloads',
 'Dropbox (CSU Fullerton)',
 'Duty, Honor, Country _ Douglas MacArthur _ May 12, 1962 _ West Point-_42_aLGkRpg.m4a',
 'env',
 'Fauci reacts to reports he needs security detail after threats-BbWEVgbIXbM.m4a',
 'Fauci says coronavirus deaths will keep 

In [108]:
# Linking the name of each audio file to the speaker
for audio_file in os.listdir('.'):
    #if 'churchill' in audio_file.lower():
    #    df_audio.at['churchill','filename'] = audio_file
    if 'threats' in audio_file.lower():
        df_audio.at['2020-04-02', 'filename'] = audio_file  
    elif 'deaths' in audio_file.lower():
        df_audio.at['2020-04-05', 'filename'] = audio_file

In [109]:
df_audio

,youtube_urls,filename
2020-04-05,https://www.youtube.com/watch?v=jJSfz7POzM8,Fauci says coronavirus deaths will keep rising...
2020-04-02,https://www.youtube.com/watch?v=BbWEVgbIXbM,Fauci reacts to reports he needs security deta...


In [12]:
import boto3

aws_access_key_id = 'A**************************'
aws_secret_access_key = 'p*******************************'
region_name = 'us-west-1'

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key,
                                region_name=region_name)

In [110]:

# Uploading files to S3
# Instantiate client
s3 = boto3.client('s3')

# List created buckets
response = s3.list_buckets()
#print(response)
#buckets = [bucket['audiofirst'] for bucket in response['us-west-1']]
buckets = [bucket['Name'] for bucket in response['Buckets']]
for bucket in buckets:
    print(bucket)

audiofirst
medicalaudio1


In [111]:
# set the bucket link to audio files
bucket_name = 'audiofirst'
client_s3 = boto3.client('s3')


In [55]:
# Uploading the files to the created bucket
for audio_file in df_audio.filename.values:
    print(audio_file)
    #client_s3.upload_file(audio_file, bucket_name,  audio_file)
    client_s3.upload_file(audio_file, 'audiofirst',  audio_file)

Fauci reacts to reports he needs security detail after threats-BbWEVgbIXbM.m4a
Fauci says coronavirus deaths will keep rising even as new U.S. cases stabilize-jJSfz7POzM8.m4a


In [112]:
df_audio

,youtube_urls,filename
2020-04-05,https://www.youtube.com/watch?v=jJSfz7POzM8,Fauci says coronavirus deaths will keep rising...
2020-04-02,https://www.youtube.com/watch?v=BbWEVgbIXbM,Fauci reacts to reports he needs security deta...


In [113]:

# Define the file URLs on the bucket using S3 convention for file paths
for index, row in df_audio.iterrows():
    bucket_location = boto3.client('s3').get_bucket_location(Bucket='audiofirst')
    object_url = f"https://{'audiofirst'}.s3.amazonaws.com/{row['filename'].replace(' ', '+')}"
    df_audio.at[index, 'url'] = object_url
    print(object_url)

https://audiofirst.s3.amazonaws.com/Fauci+says+coronavirus+deaths+will+keep+rising+even+as+new+U.S.+cases+stabilize-jJSfz7POzM8.m4a
https://audiofirst.s3.amazonaws.com/Fauci+reacts+to+reports+he+needs+security+detail+after+threats-BbWEVgbIXbM.m4a


In [115]:
df_audio

,youtube_urls,filename,url
2020-04-05,https://www.youtube.com/watch?v=jJSfz7POzM8,Fauci says coronavirus deaths will keep rising...,https://audiofirst.s3.amazonaws.com/Fauci+says...
2020-04-02,https://www.youtube.com/watch?v=BbWEVgbIXbM,Fauci reacts to reports he needs security deta...,https://audiofirst.s3.amazonaws.com/Fauci+reac...


In [116]:

# Function to start Amazon Transcribe job
def start_transcription(bucket, job_name, file_url, wait_process=True):
    client_transcribe = boto3.client('transcribe', region_name='us-west-1')
    client_transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_url},
        MediaFormat='mp4',
        LanguageCode='en-US',
        OutputBucketName=bucket)
    if wait_process:
        while True:
            status = client_transcribe.get_transcription_job(TranscriptionJobName=job_name)
            if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                break
            print("Not ready yet...")
            time.sleep(20)

        print('Transcription finished')
        return status
    
        

In [121]:

# Define today's date to use on job name
today = date.today().strftime("%d%m%Y")

# Iterate over the audio files URLs on S3 and call start_transcription
# the name in start_ & df_audio.at should be the same
for index, row in df_audio.iterrows():    
    print(f'{index}_speech_{today}', row.url)
    start_transcription(bucket_name, f'{index}_speech_{today}', row.url, wait_process=False)
    #original
    #start_transcription(bucket_name, f'{index}_speech_{today}', row.url, wait_process=False)
    df_audio.at[index, 'transcription_url'] = f"https://{bucket_name}.s3.amazonaws.com/{index}_speech_{today}.json"
    df_audio.at[index, 'json_transcription'] = f"{index}_speech_{today}.json"

2020-04-05_speech_06042020 https://audiofirst.s3.amazonaws.com/Fauci+says+coronavirus+deaths+will+keep+rising+even+as+new+U.S.+cases+stabilize-jJSfz7POzM8.m4a
2020-04-02_speech_06042020 https://audiofirst.s3.amazonaws.com/Fauci+reacts+to+reports+he+needs+security+detail+after+threats-BbWEVgbIXbM.m4a


In [122]:
# Verify Amazon Transcrive jobs that are under the status COMPLETE
client_transcribe = boto3.client('transcribe', region_name='us-west-1')
response_transcribe = client_transcribe.list_transcription_jobs(
    Status='COMPLETED')
   
response_transcribe

{'Status': 'COMPLETED',
 'NextToken': '3/Q22yZ4n/FldYqPp90dXuFrGDwH9mu7LZ4J1RaiIpumT2vQoyWUUpbJXmD+zMmcF2goLop+urShc1LxISp8yj9j3ICWcCYqriYtKtKzqdAK2Me8BrISagoFsRIp4s5MU0p/1q3UaHZpNRK2kFRCxDJloP0B1kITjpxuxx0WdAiN5hy384/nu/4Dh4XGvgOZvaObvkGJb5sjplcdPoGSbQZCnrYlURFgBRa9BRxv/YYSlnPRmZXw9jwHDp2zzBInNcruKV9NAQoLtxLXI/qdmai7cZE6yHi+2tvNrmJMAUHQaicxP5Q57DLDoxFL92q8v0fk3DSLG1z92qyfTkEMjCo8JrXLD4QZUzdbxx3DYBl3400hMfkE+RZGTw3nVuzfSd3E83tFaLtkI91y4zmLPbiNYr+k8+jYvB6rvqeCLWp3UIvCe7Dy71A2mL1Z5HHwRZWMBaS4XZRAKhrKlIkGIZNWUPDaFxBt0+ZwBpMLrrO2M3xOjhFrk8h5pPzSdWsLq5TcuOvpL8etX54rLM8tg9oDWom5tG5YGEIjteQgwrA4YZNxeu/e2vHq9bOjMYS32ar8WAAtzPpBLmWWW4VJJb09x5MerXdU6pUTXrYbrXxk9fVNvFEkfeRgvXI4yNz3hu8d9vFeCRHWptwkZPHQl07mu+SvvtWZSWj8JNgkKPE31kxH3zhunnQSoDQ6xNawNFCa8V+r+QJge/TKvoPhy861QerK4c+KGJ1EUAkL/5eMCEAT9p0aE18cL1gYy6EOvrgpcUWRlpR6F0pGWFORAkHLTz1AAn2sb9LlY+/I9+i6dCDHd7h+1zrEZNlTxZO5NhmYiKnuZZxLdvaY85JzuhU08pvM3KR6f0kti1LTjMQSw8HXrQww8xnI9hpDErYgvIAUZ+gA==',
 'TranscriptionJobSummaries': [{'TranscriptionJobNam